In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

In [2]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
    "cvxCRV" : "CRV"
}


In [3]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [4]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['tvl'] = data['tvl'] * 1000
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

In [5]:
date = datetime(2021, 7, 19)
dt = date.timestamp()
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'Sushi']

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,BEL,5.348134,8.713252e+03,Sushi,ETH,POOL
1,ETH,UMA,3.983680,4.501149e+06,Sushi,ETH,POOL
2,ETH,UWL,1.748282,1.570229e+04,Sushi,ETH,POOL
4,stXYM,stETH,0.000000,0.000000e+00,Sushi,ETH,POOL
5,ETH,LIGHT,0.134295,1.960001e+03,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1760,USDP,DUCK,0.595287,7.499317e+04,Sushi,ETH,POOL
1761,AMIS,USDC,0.589157,1.353760e+02,Sushi,ETH,POOL
1762,ETH,SAK3,0.351563,1.906976e+03,Sushi,ETH,POOL
1767,ETH,SEC,0.649223,7.272356e+04,Sushi,ETH,POOL


In [6]:
df

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ALEPH,USDC,0.890000,4.312502e+06,Raydium,SOL,POOL
1,APEX,USDC,21.520000,2.317105e+02,Raydium,SOL,POOL
2,APYS,USDC,6.610000,1.034717e+04,Raydium,SOL,POOL
3,AUSS,USDC,0.000000,1.000000e-04,Raydium,SOL,POOL
4,BLD,USDC,43.630000,7.828830e+01,Raydium,SOL,POOL
...,...,...,...,...,...,...,...
1764,ETH,0xMR,2.206443,4.002271e+03,Uniswap,ETH,POOL
1765,ETH,CHZ,15.006481,4.889787e+04,Uniswap,ETH,POOL
1766,ETH,POLS,1.861406,4.465403e+06,Uniswap,ETH,POOL
1767,ETH,SEC,0.649223,7.272356e+04,Sushi,ETH,POOL


In [7]:
import os
import re
import pandas as pd

In [8]:
def matched(regex, line):
    matched = re.match(regex, line)
    return bool(matched)

In [9]:
def get_info(block):
    pass

In [10]:
def clean_block(block):
    regexs = ['[\d]+ [\-]$', 'Total [s,S]taked:', 'NOTE: Once rewards', 'Each time you', "Staking Contract", "Found ", "Showing incentivized"
             ,"Finished reading smart", "The following farms have"]
    if len(block) > 0:
        for regex in regexs:
            if matched(regex, block[0]):
                return block[1:]
    return block
    

In [11]:
def get_single_pair1(line):
    if " " not in line:
        return [line, ""]

def get_single_pair2(line):
    if matched("[\d]+ [\-] ", line):
        return [line.split(" ")[2], ""]
    return ["", ""]

In [12]:
def get_pair2(line):
    if "-" in line:
        return line.split("-")
    return ["", ""]

In [13]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        return s
#         if all(ord(c) < 128 for c in s):
#             return s
#         else:
#             print(line)
#             return None
    except:
        return ''

def pair2(line):
#     print(line)
    if "-[" not in line:
        return ""
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
    return s
#     print(s)
#     if all(ord(c) < 128 for c in s):
#         return s
#     else:
#         print(line)
#         return ''

In [14]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

def get_single_tvl(line):
    exp = "\(*[0-9\,\.$]+\)*"
    x = re.findall(exp, line)
    return x[-1].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()

def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [15]:
def get_pair(block):
    return [pair1(block[0]) , pair2(block[0])]

def get_apr(block):
    for line in block:
        if matched("APR *:", line):
            return apr(line)
    return 0

def get_tvl(block):
    for line in block:
        if "TVL: " in line:
            return tvl(line)
    for line in block:
        if "Stacked:" in line:
            return get_single_tvl(line)
    return None

        
def get_where(block):
    return where(block[0])

In [16]:
def get_pair_info(block, where):
#     if len(block)> 0:
#         if matched('[\d]+ [\-]$', block[0]):
#             if len(block) > 1:
#                 print(len(block), "\t", block[1], "Z#####################")
#             else:
#                 print(len(block), "\t", block[0], "\t-------------------------")
#         else:
#             print(len(block), "\t", block[0])
    pair = get_pair(block)
    if pair is None or len(pair[0]) == 0:
        pair = get_pair(block[0])
    if pair is None or len(pair[0]) == 0:
        pair = get_single_pair1(block[0])
    if pair is None or len(pair[0]) == 0:
        pair = get_single_pair2(block[0])
    return [pair[0].replace("τ", "T", -1), pair[1].replace("τ", "T", -1), get_apr(block), get_tvl(block), where]

In [17]:
def get_pair_from_blocks(blocks):
    where = None
    pairs = []
    for block in blocks:
        block2 = block
        correction_loop = 0
        while correction_loop < 10:
            block = clean_block(block)
            correction_loop = correction_loop + 1
        if len(block) == 1:
            where_tmp = get_where(block)
            where = where_tmp
#             print(where)
        elif len(block) > 0 and len(block) < 25:
            pair = get_pair_info(block, where)
#             print("pair: ", pair)
            if pair is None or len(pair[0]) == 0:
                print('No Pair')
            else:
                pairs.append(pair)
        else:
            if(len(block)) > 0:
                print("Bigger Block as Test")
    return pairs
        

In [18]:
def convert_to_pair_blocks2(data_lines):
#     print(data_lines)
    blocks = []
    block = []
    index = 0
    while index < len(data_lines):
        line = data_lines[index]
        if "[%]" in line :
            tmp_block = block[:-3]
            if len(tmp_block):
                blocks.append(tmp_block)
            block = []
            block.append(data_lines[index - 2])
            block.append(data_lines[index - 1])
            block.append(line)
        elif "Exit" in line:
            tmp_block = block[:]
            if len(tmp_block):
                tmp_block.append(line)
                blocks.append(tmp_block)
            block = []
        elif "INFO" in line:
            blocks.append([line.strip()])
            block = []
        elif matched("[\d]+ [\-] *", line):
            blocks.append(block)
            block = []
        elif "Staking or unstaking also" in line:
            block.append(line)
            blocks.append(block)
            block = []
        else:
            block.append(line)
        index = index + 1
    if len(block):
        blocks.append(block)
#     print(blocks)
    return blocks
        

In [19]:
def extract_single_pairs(blocks):
    all_blocks = []
    for block in blocks:
        all_blocks.extend(convert_to_pair_blocks2(block))
    return all_blocks

In [20]:
def convert_to_pair_blocks(data_lines):
#     print(data_lines)
    blocks = []
    block = []
    index = 0
    while index < len(data_lines):
        line = data_lines[index]
        if "INFO" in line:
            blocks.append([line])
            block = []
        elif "[+]" in line:
            tmp_block = block[:-1]
            if len(tmp_block):
                blocks.append(tmp_block)
            block = []
            block.append(data_lines[index - 1])
            block.append(line)
        else:
            block.append(line)
            
        index = index + 1
    blocks.append(block)
    return extract_single_pairs(blocks)
        

In [21]:
def read_data_rows_from_files(file):
    f = open(file)
    lines = f.readlines()
    start_index = 0
    data_lines = []
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if "INFO" in line:
#             print(line.strip())
            data_lines.append(line)
        if "Reading smart contracts" in line:
            while start_index < len(lines):
                line = lines[start_index].strip()
                start_index = start_index + 1
                if len(line):
#                     print(line)
                    data_lines.append(line)
    return convert_to_pair_blocks(data_lines)
                    

In [22]:
def convert_file_to_blocks(file_name):
    if ".DS_Store" not in file_name:
#         print(file_name)
        file_blocks = read_data_rows_from_files(file_name)
        return file_blocks
    return []

In [23]:
def conver_files_to_one_df(data_dir, network=None):
    files = os.listdir(data_dir)
    dfs = pd.DataFrame([])
    for file_name in files:
        file = f"{data_dir}/{file_name}"
        print(file)
        print(dfs.shape)
        if os.path.isdir(file):
            network = file_name.upper()
            if "ALL" == network:
                network = 'ETH'
            df = conver_files_to_one_df(file, network)
            print("Dir:", network, "Total Pairs: ", df.shape)
            dfs = dfs.append(df)
            print("Dir Return, Dfs: ", dfs.shape)
        else:
            file_blocks = convert_file_to_blocks(file)
            pairs = get_pair_from_blocks(file_blocks)
            df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'apr', 'tvl', 'where',])
            df['network'] = network
            print(df.shape)
            dfs = dfs.append(df)
            print("After File Apend, Dfs: ", dfs.shape)
    return dfs.reset_index(drop=True)

In [24]:
data_dir = './out'
pair_df = conver_files_to_one_df(data_dir)

./out/ploygon
(0, 0)
./out/ploygon/_cometh.txt
(0, 0)
(50, 6)
After File Apend, Dfs:  (50, 6)
./out/ploygon/_kyber.txt
(50, 6)
(6, 6)
After File Apend, Dfs:  (56, 6)
./out/ploygon/_quick.txt
(56, 6)
(76, 6)
After File Apend, Dfs:  (132, 6)
./out/ploygon/_dark.txt
(132, 6)
(1, 6)
After File Apend, Dfs:  (133, 6)
./out/ploygon/_smartdex.txt
(133, 6)
(7, 6)
After File Apend, Dfs:  (140, 6)
Dir: PLOYGON Total Pairs:  (140, 6)
Dir Return, Dfs:  (140, 6)
./out/fantom
(140, 6)
./out/fantom/_hyperjump.txt
(0, 0)
Bigger Block as Test
(27, 6)
After File Apend, Dfs:  (27, 6)
./out/fantom/_popsicle.txt
(27, 6)
(5, 6)
After File Apend, Dfs:  (32, 6)
./out/fantom/_spirit.txt
(32, 6)
Bigger Block as Test
(26, 6)
After File Apend, Dfs:  (58, 6)
./out/fantom/_spooky.txt
(58, 6)
(22, 6)
After File Apend, Dfs:  (80, 6)
./out/fantom/_borg.txt
(80, 6)
(46, 6)
After File Apend, Dfs:  (126, 6)
Dir: FANTOM Total Pairs:  (126, 6)
Dir Return, Dfs:  (266, 6)
./out/bsc
(266, 6)
./out/bsc/_autofarm.txt
(0, 0)
No P

In [25]:
pair_df

,pair1,pair2,apr,tvl,where,network
0,WMATIC,WETH,26.86,564259.50,https://swap.cometh.io/#/stake,PLOYGON
1,LADZ,WETH,0.00,22695.18,https://swap.cometh.io/#/stake,PLOYGON
2,LADZ,MUST,0,13834.50,https://swap.cometh.io/#/stake,PLOYGON
3,MUST,pBTC,47.25,14267.74,https://swap.cometh.io/#/stake,PLOYGON
4,WBTC,pBTC,11.26,29892.35,https://swap.cometh.io/#/stake,PLOYGON
...,...,...,...,...,...,...
840,WAVAX,ETH,28.62,82596.50,https://exchange.yetiswap.app,AVAX
841,WAVAX,USDT,80.36,41245.92,https://exchange.yetiswap.app,AVAX
842,YTS,ETH,144.18,268.41,https://exchange.yetiswap.app,AVAX
843,YTS,USDT,102.46,873.73,https://exchange.yetiswap.app,AVAX


In [26]:
pair_df['apy'] = np.nan

In [27]:
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == ''].index)
pair_df[pair_df['pair1'] == '']

,pair1,pair2,apr,tvl,where,network,apy


In [28]:
def get_symbol_mapping(pair):
    if pair is not None:
        return sm[pair.strip()] if pair.strip() in sm else pair.strip()
    else:
        return ""

def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [29]:
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)

In [30]:

pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: get_symbol_mapping(x))

In [31]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr is not None and len(apr.strip()):
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy):
            return
        if apy is not None and len(apy.strip()):
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(2726, 8)
(0, 8)


In [32]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,apr,tvl,where,network,apy,type_of_agriculture
240,CREAM,,1713256.72,None,https://ftm.borgswap.exchange,FANTOM,NaN,FARM
407,BNB,,6573793244067398656.00,None,https://autofarm.network,BSC,NaN,FARM
1375,ETH,CAPY,589460.034624,374.122744,Uniswap,ETH,NaN,POOL


In [33]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,apr,tvl,where,network,apy,type_of_agriculture


In [34]:
pair_df = pair_df.drop_duplicates()
pair_df

,pair1,pair2,apr,tvl,where,network,apy,type_of_agriculture
0,WMATIC,ETH,26.86,564259.50,https://swap.cometh.io/#/stake,PLOYGON,30.800257,FARM
1,LADZ,ETH,0.00,22695.18,https://swap.cometh.io/#/stake,PLOYGON,0.000000,FARM
2,LADZ,MUST,0,13834.50,https://swap.cometh.io/#/stake,PLOYGON,0.000000,FARM
3,MUST,pBTC,47.25,14267.74,https://swap.cometh.io/#/stake,PLOYGON,60.350913,FARM
4,BTC,pBTC,11.26,29892.35,https://swap.cometh.io/#/stake,PLOYGON,11.916474,FARM
...,...,...,...,...,...,...,...,...
2721,ETH,0xMR,2.20644252,4002.270682,Uniswap,ETH,2.230896,POOL
2722,ETH,CHZ,15.00648084,48897.873562,Uniswap,ETH,16.187371,POOL
2723,ETH,POLS,1.86140556,4465403.184623,Uniswap,ETH,1.878789,POOL
2724,ETH,SEC,0.64922304,72723.563054,Sushi,ETH,0.651329,POOL


In [35]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [36]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [37]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,apr,tvl,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,0.000000,POOL
1,$BASED,ETH,5.7197484,7287.474348,Uniswap,ETH,5.886015,POOL
2,$BASED,SUSD,14.593619280000002,8777.567023,Uniswap,ETH,15.708860,POOL
3,$BASED,sUSD,0.130785,76.26718,Sushi,ETH,0.130870,POOL
4,$DG,BNB,189.89,1986849.79,https://pancakeswap.finance,BSC,564.575021,FARM
...,...,...,...,...,...,...,...,...
5305,zLOT,ETH,0.7344576,18483.175967,Sushi,ETH,0.737154,POOL
5306,zSUSHI,ZERO,0.00,27911.97,https://app.0.exchange,AVAX,0.000000,FARM
5307,zUNI,ZERO,0.00,9903.77,https://app.0.exchange,AVAX,0.000000,FARM
5308,zUSDC,ZERO,0.00,19422.07,https://app.0.exchange,AVAX,0.000000,FARM


In [38]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [39]:
summary

{'total_pairs': 2724,
 'single_coin_pairs': 69,
 'double_coin_piars': 2655,
 'after_switching_coins_total_double_coin_pairs': 5310,
 'total_pairs_in_sheet': 5379}

In [40]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [41]:
pair_df

,pair1,pair2,apr,tvl,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,0.000000,POOL
1,$BASED,ETH,5.7197484,7287.474348,Uniswap,ETH,5.886015,POOL
2,$BASED,SUSD,14.593619280000002,8777.567023,Uniswap,ETH,15.708860,POOL
3,$BASED,sUSD,0.130785,76.26718,Sushi,ETH,0.130870,POOL
4,$DG,BNB,189.89,1986849.79,https://pancakeswap.finance,BSC,564.575021,FARM
...,...,...,...,...,...,...,...,...
5374,zLOT,ETH,0.7344576,18483.175967,Sushi,ETH,0.737154,POOL
5375,zSUSHI,ZERO,0.00,27911.97,https://app.0.exchange,AVAX,0.000000,FARM
5376,zUNI,ZERO,0.00,9903.77,https://app.0.exchange,AVAX,0.000000,FARM
5377,zUSDC,ZERO,0.00,19422.07,https://app.0.exchange,AVAX,0.000000,FARM


In [42]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = date 
del pair_df['where']

In [43]:
pair_df['dex'].unique().tolist()

['Sushi',
 'Uniswap',
 'https://pancakeswap.finance',
 'https://quickswap.exchange/#/quick',
 'https://swap.cometh.io/#/stake',
 'https://1inch.exchange/#/dao/farming',
 'https://autofarm.network',
 'https://app.pangolin.exchange/#/png',
 'https://apeswap.finance',
 'https://spookyswap.finance',
 'https://swap.smartdex.app',
 'Raydium',
 'https://www.bakeryswap.org',
 'https://app.spiritswap.finance',
 'https://armor.fi',
 'https://www.arthcoin.com',
 'https://cafeswap.finance',
 'https://ftm.borgswap.exchange',
 'https://hyperjump.fi',
 'https://pandaswap.exchange',
 'https://avax.complus.exchange',
 'https://exchange.yetiswap.app',
 'https://app.0.exchange',
 'https://www.dark.build',
 'https://dmm.exchange/',
 'https://kitten.finance/',
 'https://popsicle.finance']

In [44]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,apr,tvl,network,apy,type_of_agriculture,dex,date
1456,ETH,,25.73,None,FANTOM,29.331586,FARM,https://ftm.borgswap.exchange,2021-07-19
1457,ETH,$BASED,5.7197484,7287.474348,ETH,5.886015,POOL,Uniswap,2021-07-19
1458,ETH,$DG,9.668104079999999,841763.014393,ETH,10.149488,POOL,Uniswap,2021-07-19
1459,ETH,$DG,11.68934316,96743.561607,ETH,12.397861,POOL,Sushi,2021-07-19
1460,ETH,$DG,59.23,592834.52,PLOYGON,80.727456,FARM,https://quickswap.exchange/#/quick,2021-07-19
...,...,...,...,...,...,...,...,...,...
2867,ETH,yyDAI+y...,1.60033596,1646538.539471,ETH,1.613174,POOL,Sushi,2021-07-19
2868,ETH,yyDAI+y...,0.78044772,126949.168724,ETH,0.783493,POOL,Uniswap,2021-07-19
2869,ETH,zHEGIC,0.7368379199999999,1876.208944,ETH,0.739552,POOL,Uniswap,2021-07-19
2870,ETH,zHEGIC,0.0,16.541198,ETH,0.000000,POOL,Sushi,2021-07-19


In [45]:
farm_grades = pd.read_csv('Yield_grades.csv')
farm_grades

,Farm,Website,Network,RugDoc / Indv.,Grade
0,Aave,https://aave.com,ETH,NaN,A
1,Armor,https://armor.fi,ETH,NaN,A
2,UniSwap (Pool),uniswap.org,ETH,NaN,A
3,Aave,https://aave.com,MATIC,NaN,A
4,Sushi,https://app.sushi.com,MATIC,NaN,A
...,...,...,...,...,...
761,NaN,https://app.xsigma.fi,ETH,NaN,C
762,NaN,https://app.pickle.finance/,MATIC,NaN,D
763,NaN,https://polyfunky.com/,MATIC,NaN,F
764,NaN,https://www.helmet.insure,ETH,NaN,C


In [46]:
farm_grades = farm_grades[['Website', 'Network', 'Grade']]
farm_grades = farm_grades.append({'Website': 'Sushi', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'https://app.sushi.com/farms/', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'Uniswap', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)

In [47]:
pairs_farm_risk_df = pd.merge(pair_df, farm_grades, how='left', left_on=['dex', 'network'], right_on=['Website', 'Network'])
pairs_farm_risk_df.to_csv('inhouse_farm_risk_yeild_report.csv')
pairs_farm_risk_df


,pair1,pair2,apr,tvl,network,apy,type_of_agriculture,dex,date,Website,Network,Grade
0,,sil,0.0,0.0,ETH,0.000000,POOL,Sushi,2021-07-19,Sushi,ETH,A
1,$BASED,ETH,5.7197484,7287.474348,ETH,5.886015,POOL,Uniswap,2021-07-19,Uniswap,ETH,A
2,$BASED,SUSD,14.593619280000002,8777.567023,ETH,15.708860,POOL,Uniswap,2021-07-19,Uniswap,ETH,A
3,$BASED,sUSD,0.130785,76.26718,ETH,0.130870,POOL,Sushi,2021-07-19,Sushi,ETH,A
4,$DG,BNB,189.89,1986849.79,BSC,564.575021,FARM,https://pancakeswap.finance,2021-07-19,https://pancakeswap.finance,BSC,B
...,...,...,...,...,...,...,...,...,...,...,...,...
5374,zLOT,ETH,0.7344576,18483.175967,ETH,0.737154,POOL,Sushi,2021-07-19,Sushi,ETH,A
5375,zSUSHI,ZERO,0.00,27911.97,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F
5376,zUNI,ZERO,0.00,9903.77,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F
5377,zUSDC,ZERO,0.00,19422.07,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F


In [48]:
pairs_farm_risk_df[(pairs_farm_risk_df['Website'].isna()) & (pairs_farm_risk_df['type_of_agriculture'] == 'POOL')]

,pair1,pair2,apr,tvl,network,apy,type_of_agriculture,dex,date,Website,Network,Grade


In [49]:
unmapped_dexs_df = pairs_farm_risk_df[pairs_farm_risk_df['Website'].isna()][['dex', 'network']].reset_index(drop=True)
unmapped_dexs_df = unmapped_dexs_df.drop_duplicates()
if unmapped_dexs_df.shape[0] > 1:
    unmapped_dexs_df.to_csv('unmapped_dex.csv')
unmapped_dexs_df


,dex,network
0,https://quickswap.exchange/#/quick,PLOYGON
2,https://swap.cometh.io/#/stake,PLOYGON
9,https://swap.smartdex.app,PLOYGON
31,https://www.dark.build,PLOYGON
35,https://dmm.exchange/,PLOYGON


In [50]:
pairs_farm_risk_df

,pair1,pair2,apr,tvl,network,apy,type_of_agriculture,dex,date,Website,Network,Grade
0,,sil,0.0,0.0,ETH,0.000000,POOL,Sushi,2021-07-19,Sushi,ETH,A
1,$BASED,ETH,5.7197484,7287.474348,ETH,5.886015,POOL,Uniswap,2021-07-19,Uniswap,ETH,A
2,$BASED,SUSD,14.593619280000002,8777.567023,ETH,15.708860,POOL,Uniswap,2021-07-19,Uniswap,ETH,A
3,$BASED,sUSD,0.130785,76.26718,ETH,0.130870,POOL,Sushi,2021-07-19,Sushi,ETH,A
4,$DG,BNB,189.89,1986849.79,BSC,564.575021,FARM,https://pancakeswap.finance,2021-07-19,https://pancakeswap.finance,BSC,B
...,...,...,...,...,...,...,...,...,...,...,...,...
5374,zLOT,ETH,0.7344576,18483.175967,ETH,0.737154,POOL,Sushi,2021-07-19,Sushi,ETH,A
5375,zSUSHI,ZERO,0.00,27911.97,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F
5376,zUNI,ZERO,0.00,9903.77,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F
5377,zUSDC,ZERO,0.00,19422.07,AVAX,0.000000,FARM,https://app.0.exchange,2021-07-19,https://app.0.exchange,AVAX,F
